In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error as MSE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [120]:
###各データのパス###
# 訓練用データ
train_path = './competition/' + 'train' + '.csv'
# テストデータ
test_path = './competition/' + 'test' + '.csv'
# 提出用データ
submit_path = './competition/' + 'submit_sample' + '.csv'

### 今のところ一番精度の良い前処理(ラベルエンコーディング)

In [121]:
le = LabelEncoder()

In [122]:
# 訓練用データの読み込み
train_df = pd.read_csv(train_path)

In [123]:
train_df['weather'].value_counts()

晴                 51
快晴                32
曇                 23
薄曇                22
曇時々雨              13
                  ..
曇一時晴後時々雨           1
晴後一時雨、雷を伴う         1
晴後一時雨、雷・ひょうを伴う     1
雨後曇一時晴             1
曇時々雨後晴             1
Name: weather, Length: 62, dtype: int64

In [124]:
train_df['kcal'] = train_df['kcal'].fillna(method = 'ffill')
train_df = train_df.fillna({'payday': 0.0, 'remarks':'特になし', 'event' : '特になし'})

In [125]:
le = le.fit(train_df['weather'])
train_df['weather_label'] = le.transform(train_df['weather'])
#train_df['weather_label'] = train_df['weather_label'].astype('category')

In [126]:
le_2 = le.fit(train_df['name'])
train_df['name_label'] = le_2.transform(train_df['name'])
#train_df['name_label'] = train_df['name_label'].astype('category')

In [127]:
train_df['remarks_b']  = train_df['remarks']
train_df.loc[train_df.remarks != '特になし','remarks_b']= 1
train_df.loc[train_df.remarks == '特になし','remarks_b']= 0
#train_df['remarks_b'] = train_df['remarks_b'].astype('category')
train_df['remarks_b'] = train_df['remarks_b'].astype('float') 

In [128]:
## 説明変数
X = train_df.drop(['dateid','y','name','remarks','soldout','weather'],axis = 1)
## 目的変数
y = train_df['y']

In [129]:
## 週データのダミー変数化(ワンホット)
X = pd.get_dummies(X,columns=['week'])
##13時開始お弁当持ち込み可の社内イベント'のダミー変数化(ワンホット)
X = pd.get_dummies(X,columns=['event'])
#X = X.drop(['event'],axis=1)
print(X)

      kcal  payday  precipitation  temperature  weather_label  name_label  \
0    377.0     0.0            0.0         13.9             22          98   
1    380.0     0.0            0.0         15.2             47         158   
2    390.0     0.0           65.0         13.8              0         183   
3    350.0     0.0           40.0         15.3             19         109   
4    387.0     0.0            0.0         11.4              9         185   
..     ...     ...            ...          ...            ...         ...   
332  418.0     0.0            2.5         26.0             32          52   
333  423.0     0.0           12.0         23.7             57         163   
334  463.0     0.0            5.5         26.9             44          66   
335  420.0     0.0            1.5         25.7             32         209   
336  435.0     0.0           14.0         24.7             33         124   

     remarks_b  week_Fri  week_Mon  week_Thu  week_Tue  week_Wed  \
0      

In [130]:
###特徴量重要度を測る指標の作成
X_feature = np.array(list(X.columns))

In [131]:
X = X.values
y = y.values

In [132]:
from sklearn.model_selection import train_test_split

In [133]:
## 時系列順になるようにtrain_test_split
#ホールドアウト検証
X_train,X_test, y_train,y_test = train_test_split(X, y, test_size=0.2, shuffle=False,random_state = 30)

In [134]:
model2 = RandomForestRegressor(random_state = 30)

In [135]:
model2.fit(X_train,y_train)

RandomForestRegressor(random_state=30)

In [136]:
#予測値の算出
y_pred_train = model2.predict(X_train)
y_pred_test = model2.predict(X_test)
#MSEの算出
mse_train = MSE(y_train, y_pred_train)
mse_test = MSE(y_test, y_pred_test)
#RMSEの算出
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
#RMSEの表示
print('訓練データに対しては'+ ' ' + str(rmse_train))
print('評価データに対しては'+ ' ' + str(rmse_test))

訓練データに対しては 5.559414333573457
評価データに対しては 10.556654369192364


In [175]:
#予測値の算出
y_pred_train = model[594].predict(X_train)
y_pred_test = model[594].predict(X_test)
#MSEの算出
mse_train = MSE(y_train, y_pred_train)
mse_test = MSE(y_test, y_pred_test)
#RMSEの算出
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
#RMSEの表示
print('訓練データに対しては'+ ' ' + str(rmse_train))
print('評価データに対しては'+ ' ' + str(rmse_test))

訓練データに対しては 5.510689409639844
評価データに対しては 9.9081618910282


In [196]:
test_df = pd.read_csv(test_path)
print(test_df)

         dateid week  soldout            name   kcal   remarks event  payday  \
0    2015-09-07  Mon        0       豚肉の柚子胡椒炒め  428.0       NaN   NaN     NaN   
1    2015-09-08  Tue        0         鶏肉の塩麹焼き  420.0       NaN   NaN     NaN   
2    2015-09-09  Wed        0       サバの味醂干し焼き  456.0       NaN   NaN     NaN   
3    2015-09-10  Thu        1            塩唐揚げ  439.0       NaN   NaN     1.0   
4    2015-09-11  Fri        0  カレーライス(ポークカレー)    NaN  お楽しみメニュー   NaN     NaN   
..          ...  ...      ...             ...    ...       ...   ...     ...   
139  2016-05-19  Thu        1        豚肉ザーサイ豆腐  490.0       NaN   NaN     NaN   
140  2016-05-20  Fri        1          キーマカレー  485.0       NaN  ママの会     NaN   
141  2016-05-23  Mon        0         鶏肉の照り焼き  505.0       NaN   NaN     NaN   
142  2016-05-24  Tue        1         海鮮チリソース  485.0       NaN   NaN     NaN   
143  2016-05-25  Wed        0     ぶり照り焼き・根菜添え  508.0       NaN   NaN     NaN   

     weather  precipitation  temperatur

In [197]:
test_df['kcal'] = test_df['kcal'].fillna(method = 'ffill')
test_df = test_df.fillna({'payday': 0.0, 'remarks':'特になし', 'event' : '特になし'})
le_2 = le.fit(test_df['weather'])
test_df['weather_label'] = le_2.transform(test_df['weather'])
#test_df['weather_label'] = test_df['weather_label'].astype('category')
le_1 = le.fit(test_df['name'])
test_df['name_label'] = le_1.transform(test_df['name'])
#test_df['name_label'] = test_df['name_label'].astype('category')
test_df['remarks_b']  = test_df['remarks']
test_df.loc[test_df.remarks != '特になし','remarks_b']='1'
test_df.loc[test_df.remarks == '特になし','remarks_b']='0'
test_df['remarks_b'] = test_df['remarks_b'].astype('float')
test_df = test_df.drop(['dateid'],axis =1)
### いらないものを消去
test_df = test_df.drop(['name','remarks','weather','soldout'],axis = 1)
## 週データのダミー変数化(ワンホット)
test_df = pd.get_dummies(test_df,columns=['week'])
##13時開始お弁当持ち込み可の社内イベント'のダミー変数化(ワンホット)
test_df = pd.get_dummies(test_df,columns=['event'])
#X = X.drop(['event'],axis=1)
test_df = test_df.values

In [199]:
model

[RandomForestRegressor(random_state=0),
 RandomForestRegressor(random_state=1),
 RandomForestRegressor(random_state=2),
 RandomForestRegressor(random_state=3),
 RandomForestRegressor(random_state=4),
 RandomForestRegressor(random_state=5),
 RandomForestRegressor(random_state=6),
 RandomForestRegressor(random_state=7),
 RandomForestRegressor(random_state=8),
 RandomForestRegressor(random_state=9),
 RandomForestRegressor(random_state=10),
 RandomForestRegressor(random_state=11),
 RandomForestRegressor(random_state=12),
 RandomForestRegressor(random_state=13),
 RandomForestRegressor(random_state=14),
 RandomForestRegressor(random_state=15),
 RandomForestRegressor(random_state=16),
 RandomForestRegressor(random_state=17),
 RandomForestRegressor(random_state=18),
 RandomForestRegressor(random_state=19),
 RandomForestRegressor(random_state=20),
 RandomForestRegressor(random_state=21),
 RandomForestRegressor(random_state=22),
 RandomForestRegressor(random_state=23),
 RandomForestRegressor(ran

In [206]:
pred_data = []
for i in range(500):
    pre = model[i].predict(test_df)
    pred_data.append(pre)

In [207]:
stack_pred = np.column_stack(pred_data)
meta_test_pred = meta_model.predict(stack_pred)

ValueError: X has 500 features, but LinearRegression is expecting 1000 features as input.

In [ ]:
meta_test_pred

In [180]:
submit = pd.read_csv(submit_path,header=None)
submit[1] = test_pred1
submit.to_csv("submit37.csv", index=False, header=False)

In [95]:
test_pred1 = first_model1.predict(test_df)
test_pred2 = first_model2.predict(test_df)
test_pred3 = first_model3.predict(test_df)
test_pred4 = first_model4.predict(test_df)

In [96]:
stack_test_pred = np.column_stack((test_pred1, test_pred2, test_pred3, test_pred4))

In [97]:
meta_test_pred = meta_model.predict(stack_test_pred)

In [100]:
print(meta_test_pred)

[ 72.21526399  81.90831469  71.10034056  56.29448187 105.46749605
  58.80556124  58.10720061  55.45403765  83.38072269  53.05600435
  51.12047423 119.74216534  60.1097791   57.58605411  54.85515586
  63.07502603  65.36322903  63.27900801 105.70069848  60.3720386
  62.66337692  67.54659063  62.93253121  45.07729652  57.81066062
  56.89187083  60.07955036  64.60212452 110.81782774  59.27028748
  58.98434837  63.04646277  50.14254113  62.28820601  61.87108115
  64.92062148  58.3826741  107.38601695  62.92720281  59.18520115
  60.04129677  61.2901157   49.97504004  58.56843154  57.87366404
  62.97682278  59.65876487 108.06206499  63.06306734  60.10627392
  53.40361476  49.95717082  60.8476309   61.95897436 106.25948265
  65.7524601   70.86548169  63.66666546  47.56416264  65.15648144
  66.18550015  69.10665647  58.6136323  112.15306701  65.67212946
  66.09753503  65.90199526  56.60874429  53.39812206  69.64057793
  60.66711762  65.08410765  60.56735391 114.83025607  68.81941924
  62.551450

In [101]:
submit = pd.read_csv(submit_path,header=None)
submit[1] = meta_test_pred
submit.to_csv("submit34.csv", index=False, header=False)

In [112]:
submit_l = pd.read_csv('submit9.csv',header=None)
print(submit_l.iloc[:,1])

0       70.23
1       82.85
2       76.03
3       62.09
4      117.82
        ...  
139     59.23
140     48.20
141     63.49
142     59.25
143     55.89
Name: 1, Length: 144, dtype: float64


In [147]:
model = []
for i in range(1000):
    random =  RandomForestRegressor(random_state = i)
    model.append(random)

In [152]:
model.index

<function list.index(value, start=0, stop=9223372036854775807, /)>

In [149]:
model[1]

RandomForestRegressor(random_state=1)

In [181]:
min = 15
pred = []
pred_data = []
for i in range(1000):
    model[i].fit(X_train,y_train)
    pre = model[i].predict(X_test)
    pred_data.append(pre)
    p = np.sqrt(MSE(y_test,pre))
    pred.append(p)

In [164]:
import math 

In [166]:
for i in range(1000):
    if min > pred[i]:
        min = pred[i]
print(min)

9.9081618910282


In [167]:
print(pred.index(min))

594


In [168]:
pred[594]

9.9081618910282

In [148]:
model

[RandomForestRegressor(random_state=0),
 RandomForestRegressor(random_state=1),
 RandomForestRegressor(random_state=2),
 RandomForestRegressor(random_state=3),
 RandomForestRegressor(random_state=4),
 RandomForestRegressor(random_state=5),
 RandomForestRegressor(random_state=6),
 RandomForestRegressor(random_state=7),
 RandomForestRegressor(random_state=8),
 RandomForestRegressor(random_state=9),
 RandomForestRegressor(random_state=10),
 RandomForestRegressor(random_state=11),
 RandomForestRegressor(random_state=12),
 RandomForestRegressor(random_state=13),
 RandomForestRegressor(random_state=14),
 RandomForestRegressor(random_state=15),
 RandomForestRegressor(random_state=16),
 RandomForestRegressor(random_state=17),
 RandomForestRegressor(random_state=18),
 RandomForestRegressor(random_state=19),
 RandomForestRegressor(random_state=20),
 RandomForestRegressor(random_state=21),
 RandomForestRegressor(random_state=22),
 RandomForestRegressor(random_state=23),
 RandomForestRegressor(ran

### スタッキング

In [69]:
first_model1 = RandomForestRegressor(random_state = 30)
## グリッドリサーチによる
#first_model2 = RandomForestRegressor(max_depth= 3, max_features= None, n_estimators= 500, random_state = 30)
first_model2 = RandomForestRegressor(random_state = 40)
### 決定木の数を増やす
#first_model3 = RandomForestRegressor(n_estimators = 1000,random_state=30)
first_model3 = RandomForestRegressor(random_state = 42)
#first_model4 = XGBRegressor(early_stopping_rounds=10)
first_model4 = RandomForestRegressor(random_state = 50)


In [70]:
first_model1.fit(X_train,y_train)
first_model2.fit(X_train,y_train)
first_model3.fit(X_train, y_train)
#eval_set = [(X_test, y_test)]
#first_model4.fit(X_train,y_train,eval_set=eval_set)
first_model4.fit(X_train,y_train)

RandomForestRegressor(random_state=50)

In [86]:
test_pred1 = first_model1.predict(X_test)
test_pred2 = first_model2.predict(X_test)
test_pred3 = first_model3.predict(X_test)
test_pred4 = first_model4.predict(X_test)

In [202]:
print(np.sqrt(MSE(y_test,test_pred1)))
print(np.sqrt(MSE(y_test,test_pred2)))
print(np.sqrt(MSE(y_test,test_pred3)))
print(np.sqrt(MSE(y_test,test_pred4)))

ValueError: Found input variables with inconsistent numbers of samples: [68, 144]

In [88]:
first_pred1 = first_model1.predict(X_test)
first_pred2 = first_model2.predict(X_test)
first_pred3 = first_model3.predict(X_test)
first_pred4 = first_model4.predict(X_test)

In [89]:
stack_pred = np.column_stack((first_pred1, first_pred2, first_pred3, first_pred4))

In [187]:
stack_pred = np.column_stack(pred_data)

In [194]:
pred_data

[array([ 62.35, 116.83,  60.72,  53.85,  57.52,  48.93,  55.13,  64.39,
         57.3 ,  66.52,  62.43,  61.18,  71.81,  63.03,  69.04,  64.24,
        117.08,  73.2 ,  66.84,  63.28,  51.54,  58.9 ,  63.45,  63.01,
         54.7 ,  47.93, 116.67,  62.3 ,  58.45,  66.01,  55.35,  66.4 ,
         70.83,  64.27,  61.86,  63.86, 120.11,  70.79,  55.52,  54.85,
         61.41,  73.69,  55.26,  57.32,  55.56,  54.47,  62.67,  58.31,
         57.04,  52.32, 106.83,  60.71,  56.12,  55.62,  58.67, 106.19,
         66.01,  66.29,  63.97,  67.95,  54.54,  63.  ,  63.72, 119.88,
         73.8 ,  59.05,  54.2 ,  54.41]),
 array([ 63.37, 117.82,  60.08,  55.92,  57.31,  47.59,  53.15,  64.84,
         56.24,  66.59,  62.81,  61.79,  72.37,  65.63,  70.91,  65.42,
        118.33,  72.92,  65.78,  63.8 ,  50.69,  56.29,  64.06,  63.93,
         55.21,  47.62, 119.23,  62.52,  58.62,  68.15,  53.62,  63.63,
         70.6 ,  60.9 ,  61.92,  62.78, 119.3 ,  70.54,  55.97,  56.44,
         62.25,  74.83

In [195]:
stack_pred

array([[ 62.35,  63.37,  60.81, ...,  61.46,  61.46,  60.42],
       [116.83, 117.82, 118.92, ..., 116.55, 118.43, 116.51],
       [ 60.72,  60.08,  60.41, ...,  59.68,  58.99,  59.77],
       ...,
       [ 59.05,  58.89,  59.05, ...,  59.67,  57.86,  59.83],
       [ 54.2 ,  55.66,  54.1 , ...,  55.67,  55.17,  56.58],
       [ 54.41,  55.18,  53.53, ...,  55.36,  53.41,  51.89]])

In [188]:
#meta_model = RandomForestRegressor(random_state = 70)
meta_model = LR()
meta_model.fit(stack_pred, y_test)

LinearRegression()

In [189]:
meta_test_pred = meta_model.predict(stack_pred)
print(np.sqrt(MSE(y_test,meta_test_pred)))

4.171927796403345e-13


In [190]:
print(meta_test_pred.shape)

(68,)


In [192]:
meta_test_pred

array([ 66., 125.,  64.,  47.,  53.,  50.,  38.,  68.,  64.,  84.,  56.,
        46.,  67.,  70.,  62.,  49., 114.,  67.,  62.,  68.,  54.,  62.,
        59.,  58.,  58.,  53., 133.,  59.,  67.,  73.,  58.,  65.,  67.,
        69.,  63.,  57.,  97.,  64.,  62.,  64.,  55.,  51.,  61.,  70.,
        54.,  29.,  60.,  68.,  68.,  54., 111.,  64.,  78.,  64.,  57.,
        85.,  45.,  56.,  58.,  63.,  62.,  75.,  41., 137.,  80.,  74.,
        66.,  60.])

In [191]:
print(y_test.shape)

(68,)


In [193]:
y_test

array([ 66., 125.,  64.,  47.,  53.,  50.,  38.,  68.,  64.,  84.,  56.,
        46.,  67.,  70.,  62.,  49., 114.,  67.,  62.,  68.,  54.,  62.,
        59.,  58.,  58.,  53., 133.,  59.,  67.,  73.,  58.,  65.,  67.,
        69.,  63.,  57.,  97.,  64.,  62.,  64.,  55.,  51.,  61.,  70.,
        54.,  29.,  60.,  68.,  68.,  54., 111.,  64.,  78.,  64.,  57.,
        85.,  45.,  56.,  58.,  63.,  62.,  75.,  41., 137.,  80.,  74.,
        66.,  60.])

## LightGBMを使用

In [902]:
%pip install lightgbm


     ---------------------------------------- 1.0/1.0 MB 10.8 MB/s eta 0:00:00


In [38]:
import lightgbm as lgb
#model = lgb.LGBMRegressor(boosting_type='gbdt', max_depth=5,random_state=20)

In [79]:
# 学習用
lgb_train = lgb.Dataset(X_train, y_train,
                        free_raw_data=False)
    # 検証用
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train,
                         free_raw_data=False)
    
# パラメータを設定
params = {'boosting_type': 'gbdt',        # 勾配ブースティング
          'objective': 'regression',      # 目的関数：回帰
          'metric': 'rmse',               # 分類モデルの性能を測る指標
          'learning_rate': 0.1}          # 学習率（初期値0.1）
    
# 学習
model = lgb.train(params,                            # 上記で設定したパラメータ
                  lgb_train,                         # 使用するデータセット
                  num_boost_round=1000,              # 学習の回数
                  valid_sets= lgb_test,  # モデル検証のデータセット
                  early_stopping_rounds=20,          # アーリーストッピング
                  verbose_eval=False)                    # 学習の経過の非表示
    
# 予測
pred = model.predict(X_test)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000172 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 274
[LightGBM] [Info] Number of data points in the train set: 269, number of used features: 13
[LightGBM] [Info] Start training from score 64.914498
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

C:\Users\baba\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\baba\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


In [80]:
mse_test = MSE(y_test, pred)
rmse_test = np.sqrt(mse_test)
print(rmse_test)

14.577450477385502


### LightGBMによる特徴量重要度の表示

In [62]:
f_importance = np.array(model.feature_importance()) # 特徴量重要度の算出
f_importance = f_importance / np.sum(f_importance)  # 正規化(必要ない場合はコメントアウト)
df_importance = pd.DataFrame({'feature':X_feature, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)

,feature,importance
4,temperature,0.384615
1,kcal,0.320513
3,precipitation,0.068376
7,remarks_b,0.059829
8,week_Fri,0.059829
0,soldout,0.040598
10,week_Thu,0.025641
9,week_Mon,0.012821
12,week_Wed,0.012821
15,event_特になし,0.008547


### XGBoost

In [944]:
%pip install xgboost

     -------------------------------------- 125.4/125.4 MB 6.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [120]:
from xgboost import XGBRegressor
model1 = XGBRegressor(early_stopping_rounds=10)
model2 = XGBRegressor(n_estimators=1000, n_estimatorsearly_stopping_rounds=10)
model3 = XGBRegressor(n_estimators=10000, n_estimatorsearly_stopping_rounds=10)

In [121]:
eval_set = [(X_test, y_test)]
model1.fit(X_train, y_train, eval_set=eval_set, verbose=True)

[0]	validation_0-rmse:50.08774
[1]	validation_0-rmse:36.82846
[2]	validation_0-rmse:27.57049
[3]	validation_0-rmse:21.33461
[4]	validation_0-rmse:17.35333
[5]	validation_0-rmse:14.90384
[6]	validation_0-rmse:13.35047
[7]	validation_0-rmse:12.79917
[8]	validation_0-rmse:12.32610
[9]	validation_0-rmse:11.90955
[10]	validation_0-rmse:11.79202
[11]	validation_0-rmse:11.67444
[12]	validation_0-rmse:11.42428
[13]	validation_0-rmse:11.47132
[14]	validation_0-rmse:11.37174
[15]	validation_0-rmse:11.42806
[16]	validation_0-rmse:11.57391
[17]	validation_0-rmse:11.58008
[18]	validation_0-rmse:11.53158
[19]	validation_0-rmse:11.60572
[20]	validation_0-rmse:11.55143
[21]	validation_0-rmse:11.56431
[22]	validation_0-rmse:11.78972
[23]	validation_0-rmse:11.74322
[24]	validation_0-rmse:11.78750


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=10, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [122]:
y_pred = model1.predict(X_test)
mse_test = MSE(y_test, y_pred)
rmse_test = np.sqrt(mse_test)
print(rmse_test)

11.371741602723409


In [123]:
eval_set = [(X_test, y_test)]
model2.fit(X_train, y_train, eval_set=eval_set, verbose=True)
y_pred = model2.predict(X_test)
mse_test = MSE(y_test, y_pred)
rmse_test = np.sqrt(mse_test)
print(rmse_test)

[18:59:50] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "n_estimatorsearly_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:50.08774
[1]	validation_0-rmse:36.82846
[2]	validation_0-rmse:27.57049
[3]	validation_0-rmse:21.33461
[4]	validation_0-rmse:17.35333
[5]	validation_0-rmse:14.90384
[6]	validation_0-rmse:13.35047
[7]	validation_0-rmse:12.79917
[8]	validation_0-rmse:12.32610
[9]	validation_0-rmse:11.90955
[10]	validation_0-rmse:11.79202
[11]	validation_0-rmse:11.67444
[12]	validation_0-rmse:11.42428
[13]	validation_0-rmse:11.47132
[14]	validation_0-rmse:11.37174
[15]	validation_0-rmse:11.42806
[16]	validation_0-rmse:11.57391
[17]	validation_0-rmse:1

[487]	validation_0-rmse:12.19302
[488]	validation_0-rmse:12.19302
[489]	validation_0-rmse:12.19302
[490]	validation_0-rmse:12.19302
[491]	validation_0-rmse:12.19302
[492]	validation_0-rmse:12.19302
[493]	validation_0-rmse:12.19302
[494]	validation_0-rmse:12.19302
[495]	validation_0-rmse:12.19302
[496]	validation_0-rmse:12.19302
[497]	validation_0-rmse:12.19302
[498]	validation_0-rmse:12.19302
[499]	validation_0-rmse:12.19302
[500]	validation_0-rmse:12.19302
[501]	validation_0-rmse:12.19302
[502]	validation_0-rmse:12.19302
[503]	validation_0-rmse:12.19302
[504]	validation_0-rmse:12.19302
[505]	validation_0-rmse:12.19302
[506]	validation_0-rmse:12.19302
[507]	validation_0-rmse:12.19302
[508]	validation_0-rmse:12.19302
[509]	validation_0-rmse:12.19302
[510]	validation_0-rmse:12.19302
[511]	validation_0-rmse:12.19302
[512]	validation_0-rmse:12.19302
[513]	validation_0-rmse:12.19302
[514]	validation_0-rmse:12.19302
[515]	validation_0-rmse:12.19302
[516]	validation_0-rmse:12.19302
[517]	vali

[736]	validation_0-rmse:12.19302
[737]	validation_0-rmse:12.19302
[738]	validation_0-rmse:12.19302
[739]	validation_0-rmse:12.19302
[740]	validation_0-rmse:12.19302
[741]	validation_0-rmse:12.19302
[742]	validation_0-rmse:12.19302
[743]	validation_0-rmse:12.19302
[744]	validation_0-rmse:12.19302
[745]	validation_0-rmse:12.19302
[746]	validation_0-rmse:12.19302
[747]	validation_0-rmse:12.19302
[748]	validation_0-rmse:12.19302
[749]	validation_0-rmse:12.19302
[750]	validation_0-rmse:12.19302
[751]	validation_0-rmse:12.19302
[752]	validation_0-rmse:12.19302
[753]	validation_0-rmse:12.19302
[754]	validation_0-rmse:12.19302
[755]	validation_0-rmse:12.19302
[756]	validation_0-rmse:12.19302
[757]	validation_0-rmse:12.19302
[758]	validation_0-rmse:12.19302
[759]	validation_0-rmse:12.19302
[760]	validation_0-rmse:12.19302
[761]	validation_0-rmse:12.19302
[762]	validation_0-rmse:12.19302
[763]	validation_0-rmse:12.19302
[764]	validation_0-rmse:12.19302
[765]	validation_0-rmse:12.19302
[766]	vali

[985]	validation_0-rmse:12.19302
[986]	validation_0-rmse:12.19302
[987]	validation_0-rmse:12.19302
[988]	validation_0-rmse:12.19302
[989]	validation_0-rmse:12.19302
[990]	validation_0-rmse:12.19302
[991]	validation_0-rmse:12.19302
[992]	validation_0-rmse:12.19302
[993]	validation_0-rmse:12.19302
[994]	validation_0-rmse:12.19302
[995]	validation_0-rmse:12.19302
[996]	validation_0-rmse:12.19302
[997]	validation_0-rmse:12.19302
[998]	validation_0-rmse:12.19302
[999]	validation_0-rmse:12.19302
12.193015208711548


In [124]:
eval_set = [(X_test, y_test)]
model3.fit(X_train, y_train, eval_set=eval_set, verbose=True)
y_pred = model3.predict(X_test)
mse_test = MSE(y_test, y_pred)
rmse_test = np.sqrt(mse_test)
print(rmse_test)

[19:00:04] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "n_estimatorsearly_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:50.08774
[1]	validation_0-rmse:36.82846
[2]	validation_0-rmse:27.57049
[3]	validation_0-rmse:21.33461
[4]	validation_0-rmse:17.35333
[5]	validation_0-rmse:14.90384
[6]	validation_0-rmse:13.35047
[7]	validation_0-rmse:12.79917
[8]	validation_0-rmse:12.32610
[9]	validation_0-rmse:11.90955
[10]	validation_0-rmse:11.79202
[11]	validation_0-rmse:11.67444
[12]	validation_0-rmse:11.42428
[13]	validation_0-rmse:11.47132
[14]	validation_0-rmse:11.37174
[15]	validation_0-rmse:11.42806
[16]	validation_0-rmse:11.57391
[17]	validation_0-rmse:1

[239]	validation_0-rmse:12.19302
[240]	validation_0-rmse:12.19302
[241]	validation_0-rmse:12.19302
[242]	validation_0-rmse:12.19302
[243]	validation_0-rmse:12.19302
[244]	validation_0-rmse:12.19302
[245]	validation_0-rmse:12.19302
[246]	validation_0-rmse:12.19302
[247]	validation_0-rmse:12.19302
[248]	validation_0-rmse:12.19302
[249]	validation_0-rmse:12.19302
[250]	validation_0-rmse:12.19302
[251]	validation_0-rmse:12.19302
[252]	validation_0-rmse:12.19302
[253]	validation_0-rmse:12.19302
[254]	validation_0-rmse:12.19302
[255]	validation_0-rmse:12.19302
[256]	validation_0-rmse:12.19302
[257]	validation_0-rmse:12.19302
[258]	validation_0-rmse:12.19302
[259]	validation_0-rmse:12.19302
[260]	validation_0-rmse:12.19302
[261]	validation_0-rmse:12.19302
[262]	validation_0-rmse:12.19302
[263]	validation_0-rmse:12.19302
[264]	validation_0-rmse:12.19302
[265]	validation_0-rmse:12.19302
[266]	validation_0-rmse:12.19302
[267]	validation_0-rmse:12.19302
[268]	validation_0-rmse:12.19302
[269]	vali

[488]	validation_0-rmse:12.19302
[489]	validation_0-rmse:12.19302
[490]	validation_0-rmse:12.19302
[491]	validation_0-rmse:12.19302
[492]	validation_0-rmse:12.19302
[493]	validation_0-rmse:12.19302
[494]	validation_0-rmse:12.19302
[495]	validation_0-rmse:12.19302
[496]	validation_0-rmse:12.19302
[497]	validation_0-rmse:12.19302
[498]	validation_0-rmse:12.19302
[499]	validation_0-rmse:12.19302
[500]	validation_0-rmse:12.19302
[501]	validation_0-rmse:12.19302
[502]	validation_0-rmse:12.19302
[503]	validation_0-rmse:12.19302
[504]	validation_0-rmse:12.19302
[505]	validation_0-rmse:12.19302
[506]	validation_0-rmse:12.19302
[507]	validation_0-rmse:12.19302
[508]	validation_0-rmse:12.19302
[509]	validation_0-rmse:12.19302
[510]	validation_0-rmse:12.19302
[511]	validation_0-rmse:12.19302
[512]	validation_0-rmse:12.19302
[513]	validation_0-rmse:12.19302
[514]	validation_0-rmse:12.19302
[515]	validation_0-rmse:12.19302
[516]	validation_0-rmse:12.19302
[517]	validation_0-rmse:12.19302
[518]	vali

[737]	validation_0-rmse:12.19302
[738]	validation_0-rmse:12.19302
[739]	validation_0-rmse:12.19302
[740]	validation_0-rmse:12.19302
[741]	validation_0-rmse:12.19302
[742]	validation_0-rmse:12.19302
[743]	validation_0-rmse:12.19302
[744]	validation_0-rmse:12.19302
[745]	validation_0-rmse:12.19302
[746]	validation_0-rmse:12.19302
[747]	validation_0-rmse:12.19302
[748]	validation_0-rmse:12.19302
[749]	validation_0-rmse:12.19302
[750]	validation_0-rmse:12.19302
[751]	validation_0-rmse:12.19302
[752]	validation_0-rmse:12.19302
[753]	validation_0-rmse:12.19302
[754]	validation_0-rmse:12.19302
[755]	validation_0-rmse:12.19302
[756]	validation_0-rmse:12.19302
[757]	validation_0-rmse:12.19302
[758]	validation_0-rmse:12.19302
[759]	validation_0-rmse:12.19302
[760]	validation_0-rmse:12.19302
[761]	validation_0-rmse:12.19302
[762]	validation_0-rmse:12.19302
[763]	validation_0-rmse:12.19302
[764]	validation_0-rmse:12.19302
[765]	validation_0-rmse:12.19302
[766]	validation_0-rmse:12.19302
[767]	vali

[986]	validation_0-rmse:12.19302
[987]	validation_0-rmse:12.19302
[988]	validation_0-rmse:12.19302
[989]	validation_0-rmse:12.19302
[990]	validation_0-rmse:12.19302
[991]	validation_0-rmse:12.19302
[992]	validation_0-rmse:12.19302
[993]	validation_0-rmse:12.19302
[994]	validation_0-rmse:12.19302
[995]	validation_0-rmse:12.19302
[996]	validation_0-rmse:12.19302
[997]	validation_0-rmse:12.19302
[998]	validation_0-rmse:12.19302
[999]	validation_0-rmse:12.19302
[1000]	validation_0-rmse:12.19302
[1001]	validation_0-rmse:12.19302
[1002]	validation_0-rmse:12.19302
[1003]	validation_0-rmse:12.19302
[1004]	validation_0-rmse:12.19302
[1005]	validation_0-rmse:12.19302
[1006]	validation_0-rmse:12.19302
[1007]	validation_0-rmse:12.19302
[1008]	validation_0-rmse:12.19302
[1009]	validation_0-rmse:12.19302
[1010]	validation_0-rmse:12.19302
[1011]	validation_0-rmse:12.19302
[1012]	validation_0-rmse:12.19302
[1013]	validation_0-rmse:12.19302
[1014]	validation_0-rmse:12.19302
[1015]	validation_0-rmse:12.

...

[8938]	validation_0-rmse:12.19302
[8939]	validation_0-rmse:12.19302
[8940]	validation_0-rmse:12.19302
[8941]	validation_0-rmse:12.19302
[8942]	validation_0-rmse:12.19302
[8943]	validation_0-rmse:12.19302
[8944]	validation_0-rmse:12.19302
[8945]	validation_0-rmse:12.19302
[8946]	validation_0-rmse:12.19302
[8947]	validation_0-rmse:12.19302
[8948]	validation_0-rmse:12.19302
[8949]	validation_0-rmse:12.19302
[8950]	validation_0-rmse:12.19302
[8951]	validation_0-rmse:12.19302
[8952]	validation_0-rmse:12.19302
[8953]	validation_0-rmse:12.19302
[8954]	validation_0-rmse:12.19302
[8955]	validation_0-rmse:12.19302
[8956]	validation_0-rmse:12.19302
[8957]	validation_0-rmse:12.19302
[8958]	validation_0-rmse:12.19302
[8959]	validation_0-rmse:12.19302
[8960]	validation_0-rmse:12.19302
[8961]	validation_0-rmse:12.19302
[8962]	validation_0-rmse:12.19302
[8963]	validation_0-rmse:12.19302
[8964]	validation_0-rmse:12.19302
[8965]	validation_0-rmse:12.19302
[8966]	validation_0-rmse:12.19302
[8967]	validat

[9179]	validation_0-rmse:12.19302
[9180]	validation_0-rmse:12.19302
[9181]	validation_0-rmse:12.19302
[9182]	validation_0-rmse:12.19302
[9183]	validation_0-rmse:12.19302
[9184]	validation_0-rmse:12.19302
[9185]	validation_0-rmse:12.19302
[9186]	validation_0-rmse:12.19302
[9187]	validation_0-rmse:12.19302
[9188]	validation_0-rmse:12.19302
[9189]	validation_0-rmse:12.19302
[9190]	validation_0-rmse:12.19302
[9191]	validation_0-rmse:12.19302
[9192]	validation_0-rmse:12.19302
[9193]	validation_0-rmse:12.19302
[9194]	validation_0-rmse:12.19302
[9195]	validation_0-rmse:12.19302
[9196]	validation_0-rmse:12.19302
[9197]	validation_0-rmse:12.19302
[9198]	validation_0-rmse:12.19302
[9199]	validation_0-rmse:12.19302
[9200]	validation_0-rmse:12.19302
[9201]	validation_0-rmse:12.19302
[9202]	validation_0-rmse:12.19302
[9203]	validation_0-rmse:12.19302
[9204]	validation_0-rmse:12.19302
[9205]	validation_0-rmse:12.19302
[9206]	validation_0-rmse:12.19302
[9207]	validation_0-rmse:12.19302
[9208]	validat

[9420]	validation_0-rmse:12.19302
[9421]	validation_0-rmse:12.19302
[9422]	validation_0-rmse:12.19302
[9423]	validation_0-rmse:12.19302
[9424]	validation_0-rmse:12.19302
[9425]	validation_0-rmse:12.19302
[9426]	validation_0-rmse:12.19302
[9427]	validation_0-rmse:12.19302
[9428]	validation_0-rmse:12.19302
[9429]	validation_0-rmse:12.19302
[9430]	validation_0-rmse:12.19302
[9431]	validation_0-rmse:12.19302
[9432]	validation_0-rmse:12.19302
[9433]	validation_0-rmse:12.19302
[9434]	validation_0-rmse:12.19302
[9435]	validation_0-rmse:12.19302
[9436]	validation_0-rmse:12.19302
[9437]	validation_0-rmse:12.19302
[9438]	validation_0-rmse:12.19302
[9439]	validation_0-rmse:12.19302
[9440]	validation_0-rmse:12.19302
[9441]	validation_0-rmse:12.19302
[9442]	validation_0-rmse:12.19302
[9443]	validation_0-rmse:12.19302
[9444]	validation_0-rmse:12.19302
[9445]	validation_0-rmse:12.19302
[9446]	validation_0-rmse:12.19302
[9447]	validation_0-rmse:12.19302
[9448]	validation_0-rmse:12.19302
[9449]	validat

[9661]	validation_0-rmse:12.19302
[9662]	validation_0-rmse:12.19302
[9663]	validation_0-rmse:12.19302
[9664]	validation_0-rmse:12.19302
[9665]	validation_0-rmse:12.19302
[9666]	validation_0-rmse:12.19302
[9667]	validation_0-rmse:12.19302
[9668]	validation_0-rmse:12.19302
[9669]	validation_0-rmse:12.19302
[9670]	validation_0-rmse:12.19302
[9671]	validation_0-rmse:12.19302
[9672]	validation_0-rmse:12.19302
[9673]	validation_0-rmse:12.19302
[9674]	validation_0-rmse:12.19302
[9675]	validation_0-rmse:12.19302
[9676]	validation_0-rmse:12.19302
[9677]	validation_0-rmse:12.19302
[9678]	validation_0-rmse:12.19302
[9679]	validation_0-rmse:12.19302
[9680]	validation_0-rmse:12.19302
[9681]	validation_0-rmse:12.19302
[9682]	validation_0-rmse:12.19302
[9683]	validation_0-rmse:12.19302
[9684]	validation_0-rmse:12.19302
[9685]	validation_0-rmse:12.19302
[9686]	validation_0-rmse:12.19302
[9687]	validation_0-rmse:12.19302
[9688]	validation_0-rmse:12.19302
[9689]	validation_0-rmse:12.19302
[9690]	validat

[9902]	validation_0-rmse:12.19302
[9903]	validation_0-rmse:12.19302
[9904]	validation_0-rmse:12.19302
[9905]	validation_0-rmse:12.19302
[9906]	validation_0-rmse:12.19302
[9907]	validation_0-rmse:12.19302
[9908]	validation_0-rmse:12.19302
[9909]	validation_0-rmse:12.19302
[9910]	validation_0-rmse:12.19302
[9911]	validation_0-rmse:12.19302
[9912]	validation_0-rmse:12.19302
[9913]	validation_0-rmse:12.19302
[9914]	validation_0-rmse:12.19302
[9915]	validation_0-rmse:12.19302
[9916]	validation_0-rmse:12.19302
[9917]	validation_0-rmse:12.19302
[9918]	validation_0-rmse:12.19302
[9919]	validation_0-rmse:12.19302
[9920]	validation_0-rmse:12.19302
[9921]	validation_0-rmse:12.19302
[9922]	validation_0-rmse:12.19302
[9923]	validation_0-rmse:12.19302
[9924]	validation_0-rmse:12.19302
[9925]	validation_0-rmse:12.19302
[9926]	validation_0-rmse:12.19302
[9927]	validation_0-rmse:12.19302
[9928]	validation_0-rmse:12.19302
[9929]	validation_0-rmse:12.19302
[9930]	validation_0-rmse:12.19302
[9931]	validat

### ランダムフォレスト

In [114]:
params = {"n_estimators": [10,20,30,40, 50,100, 300, 500,1000,10000], 
          "max_depth": [1,3,5,10,15, 20,None], 
          }

In [248]:
model2 = RandomForestRegressor(random_state = 30)

In [230]:
## 時系列順になるようにtrain_test_split
X_train_val,X_test, y_train_val,y_test = train_test_split(X, y, test_size=0.2, shuffle=False,random_state = 1030)
X_train, X_val, y_train,y_val = train_test_split(X_train_val,y_train_val,test_size=0.2,shuffle=False,random_state = 0)

In [232]:
best_val_score = 100
best_param = {}

In [192]:
forest

RandomForestRegressor(max_depth=5, n_estimators=20, random_state=30)

### グリッドサーチライブラリを使った場合

In [116]:
model4 = GridSearchCV(model2, params,scoring="neg_mean_squared_error")

In [117]:
model4.fit(X_train,y_train)
print(model4.best_params_)

{'max_depth': 3, 'n_estimators': 500}


In [119]:
forest2 = model4.best_estimator_
#予測値の算出
y_pred_train = forest2.predict(X_train)
y_pred_test = forest2.predict(X_test)
#MSEの算出
mse_train = MSE(y_train, y_pred_train)
mse_test = MSE(y_test, y_pred_test)
#RMSEの算出
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
#RMSEの表示
print('訓練データに対しては'+ ' ' + str(rmse_train))
print('評価データに対しては'+ ' ' + str(rmse_test))

訓練データに対しては 13.334303446193614
評価データに対しては 11.164703529559107


In [249]:
model2.fit(X_train,y_train)

RandomForestRegressor(random_state=30)

In [250]:
#予測値の算出
y_pred_train = model2.predict(X_train)
y_pred_test = model2.predict(X_test)
#MSEの算出
mse_train = MSE(y_train, y_pred_train)
mse_test = MSE(y_test, y_pred_test)
#RMSEの算出
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
#RMSEの表示
print('訓練データに対しては'+ ' ' + str(rmse_train))
print('評価データに対しては'+ ' ' + str(rmse_test))

訓練データに対しては 5.697133760541511
評価データに対しては 10.471793977998567


In [ ]:
#予測値の算出
y_pred_train = model2.predict(X_train)
y_pred_test = model2.predict(X_test)
#MSEの算出
mse_train = MSE(y_train, y_pred_train)
mse_test = MSE(y_test, y_pred_test)
#RMSEの算出
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
#RMSEの表示
print('訓練データに対しては'+ ' ' + str(rmse_train))
print('評価データに対しては'+ ' ' + str(rmse_test))

In [35]:
feature_importance = pd.DataFrame({'feature':X_feature,'importances': model2.feature_importances_}).sort_values(by="importances", ascending=False)

In [36]:
### 特記事項の重要度が高い
feature_importance

,feature,importances
7,remarks_b,0.215918
8,week_Fri,0.170671
6,name_label,0.145048
4,temperature,0.137768
1,kcal,0.131259
5,weather_label,0.080054
3,precipitation,0.047052
0,soldout,0.014448
9,week_Mon,0.013758
15,event_特になし,0.011682


### テストデータに関する検証

In [179]:
test_df = pd.read_csv(test_path)
print(test_df)

         dateid week  soldout            name   kcal   remarks event  payday  \
0    2015-09-07  Mon        0       豚肉の柚子胡椒炒め  428.0       NaN   NaN     NaN   
1    2015-09-08  Tue        0         鶏肉の塩麹焼き  420.0       NaN   NaN     NaN   
2    2015-09-09  Wed        0       サバの味醂干し焼き  456.0       NaN   NaN     NaN   
3    2015-09-10  Thu        1            塩唐揚げ  439.0       NaN   NaN     1.0   
4    2015-09-11  Fri        0  カレーライス(ポークカレー)    NaN  お楽しみメニュー   NaN     NaN   
..          ...  ...      ...             ...    ...       ...   ...     ...   
139  2016-05-19  Thu        1        豚肉ザーサイ豆腐  490.0       NaN   NaN     NaN   
140  2016-05-20  Fri        1          キーマカレー  485.0       NaN  ママの会     NaN   
141  2016-05-23  Mon        0         鶏肉の照り焼き  505.0       NaN   NaN     NaN   
142  2016-05-24  Tue        1         海鮮チリソース  485.0       NaN   NaN     NaN   
143  2016-05-25  Wed        0     ぶり照り焼き・根菜添え  508.0       NaN   NaN     NaN   

     weather  precipitation  temperatur

In [180]:
test_df.describe()

,soldout,kcal,payday,precipitation,temperature
count,144.00000,128.000000,8.0,144.000000,144.000000
mean,0.31250,458.125000,1.0,6.440972,12.493056
std,0.46513,38.367063,0.0,18.772143,5.816178
min,0.00000,415.000000,1.0,0.000000,2.800000
25%,0.00000,428.000000,1.0,0.000000,7.500000
50%,0.00000,445.000000,1.0,0.000000,11.600000
75%,1.00000,480.000000,1.0,1.625000,17.225000
max,1.00000,589.000000,1.0,156.500000,23.900000


In [181]:
train_df.describe()

,y,soldout,kcal,payday,precipitation,temperature,weather_label,name_label,remarks_b
count,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000,337.000000
mean,65.192878,0.462908,419.189911,0.050445,5.376855,18.520772,25.635015,108.148368,0.145401
std,20.645712,0.499364,24.745120,0.219187,14.342239,7.708158,18.436353,59.888751,0.353028
min,25.000000,0.000000,324.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000
25%,53.000000,0.000000,410.000000,0.000000,0.000000,12.300000,5.000000,57.000000,0.000000
50%,62.000000,0.000000,423.000000,0.000000,0.000000,20.300000,24.000000,105.000000,0.000000
75%,72.000000,1.000000,435.000000,0.000000,4.500000,24.000000,41.000000,163.000000,0.000000
max,137.000000,1.000000,463.000000,1.000000,123.500000,31.500000,61.000000,212.000000,1.000000


In [182]:
test_df['kcal'] = test_df['kcal'].fillna(method = 'ffill')
test_df = test_df.fillna({'payday': 0.0, 'remarks':'特になし', 'event' : '特になし'})

In [183]:
le_2 = le.fit(test_df['weather'])
test_df['weather_label'] = le_2.transform(test_df['weather'])
#test_df['weather_label'] = test_df['weather_label'].astype('category')

In [184]:
le_1 = le.fit(test_df['name'])
test_df['name_label'] = le_1.transform(test_df['name'])
#test_df['name_label'] = test_df['name_label'].astype('category')

In [185]:
test_df['remarks_b']  = test_df['remarks']
test_df.loc[test_df.remarks != '特になし','remarks_b']='1'
test_df.loc[test_df.remarks == '特になし','remarks_b']='0'
test_df['remarks_b'] = test_df['remarks_b'].astype('float')

In [186]:
test_df = test_df.drop(['dateid'],axis =1)

In [187]:
### いらないものを消去
test_df = test_df.drop(['name','remarks','weather'],axis = 1)

In [188]:
## 週データのダミー変数化(ワンホット)
test_df = pd.get_dummies(test_df,columns=['week'])
##13時開始お弁当持ち込み可の社内イベント'のダミー変数化(ワンホット)
test_df = pd.get_dummies(test_df,columns=['event'])
#X = X.drop(['event'],axis=1)

In [189]:
test_df = test_df.values

In [194]:
test_pred1 = first_model1.predict(test_df)

In [195]:
first_model1

RandomForestRegressor(random_state=30)

In [196]:
submit = pd.read_csv(submit_path,header=None)
submit[1] = test_pred1
submit.to_csv("submit24.csv", index=False, header=False)

### 前処理違う方法で！

#### メインメニューをカウントエンコーディング，天気を晴，快晴，曇の3つに

In [974]:
# 訓練用データの読み込み
train_df = pd.read_csv(train_path)

In [975]:
train_df['kcal'] = train_df['kcal'].fillna(method = 'ffill')
train_df = train_df.fillna({'payday': 0.0, 'remarks':'特になし', 'event' : '特になし'})

In [976]:
train_df['weather_l'] = train_df['weather']
train_df.loc[~((train_df['weather'] == '晴')|(train_df['weather'] == '快晴')|(train_df['weather'] == '曇')) , 'weather_l'] = 'その他'

In [ ]:
le_2 = le.fit(train_df['name'])
train_df['name_label'] = le_2.transform(train_df['name'])

In [977]:
train_df['weather_l'].head(10)

0    その他
1    その他
2    その他
3    その他
4    その他
5     快晴
6    その他
7      晴
8      晴
9    その他
Name: weather_l, dtype: object

In [978]:
## 特記事項の有無
train_df['remarks_b']  = train_df['remarks']
train_df.loc[train_df.remarks != '特になし','remarks_b']= 1
train_df.loc[train_df.remarks == '特になし','remarks_b']= 0

In [979]:
train_df['remarks_b'] = train_df['remarks_b'].astype('float')

In [980]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   dateid         337 non-null    object 
 1   y              337 non-null    float64
 2   week           337 non-null    object 
 3   soldout        337 non-null    int64  
 4   name           337 non-null    object 
 5   kcal           337 non-null    float64
 6   remarks        337 non-null    object 
 7   event          337 non-null    object 
 8   payday         337 non-null    float64
 9   weather        337 non-null    object 
 10  precipitation  337 non-null    float64
 11  temperature    337 non-null    float64
 12  weather_l      337 non-null    object 
 13  remarks_b      337 non-null    float64
dtypes: float64(6), int64(1), object(7)
memory usage: 37.0+ KB


In [981]:
## 説明変数
X = train_df.drop(['dateid','y','name','remarks','weather'],axis = 1)
## 目的変数
y = train_df['y']

In [982]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   week           337 non-null    object 
 1   soldout        337 non-null    int64  
 2   kcal           337 non-null    float64
 3   event          337 non-null    object 
 4   payday         337 non-null    float64
 5   precipitation  337 non-null    float64
 6   temperature    337 non-null    float64
 7   weather_l      337 non-null    object 
 8   remarks_b      337 non-null    float64
dtypes: float64(5), int64(1), object(3)
memory usage: 23.8+ KB


In [983]:
## 週データのダミー変数化(ワンホット)
X = pd.get_dummies(X,columns=['week'])
##13時開始お弁当持ち込み可の社内イベント'のダミー変数化(ワンホット)
X = pd.get_dummies(X,columns=['event'])
X = pd.get_dummies(X,columns=['weather_l'])

In [984]:
X

,soldout,kcal,payday,precipitation,temperature,remarks_b,week_Fri,week_Mon,week_Thu,week_Tue,week_Wed,event_キャリアアップ支援セミナー,event_ママの会,event_特になし,weather_l_その他,weather_l_快晴,weather_l_晴,weather_l_曇
0,1,377.0,0.0,0.0,13.9,0.0,0,0,0,1,0,0,0,1,1,0,0,0
1,0,380.0,0.0,0.0,15.2,0.0,0,0,0,0,1,0,0,1,1,0,0,0
2,1,390.0,0.0,65.0,13.8,0.0,0,0,1,0,0,0,0,1,1,0,0,0
3,1,350.0,0.0,40.0,15.3,0.0,1,0,0,0,0,0,0,1,1,0,0,0
4,0,387.0,0.0,0.0,11.4,0.0,0,1,0,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,0,418.0,0.0,2.5,26.0,1.0,1,0,0,0,0,0,0,1,1,0,0,0
333,1,423.0,0.0,12.0,23.7,1.0,0,0,0,1,0,0,0,1,1,0,0,0
334,0,463.0,0.0,5.5,26.9,0.0,0,0,0,0,1,0,0,1,1,0,0,0
335,1,420.0,0.0,1.5,25.7,0.0,0,0,1,0,0,0,0,1,1,0,0,0


In [993]:
## 時系列順になるようにtrain_test_split
X_train,X_test, y_train,y_test = train_test_split(X, y, test_size=0.2, shuffle=False,random_state = 1030)

In [994]:
model2.fit(X_train,y_train)

RandomForestRegressor(random_state=30)

In [995]:
#予測値の算出
y_pred_train = model2.predict(X_train)
y_pred_test = model2.predict(X_test)
#MSEの算出
mse_train = MSE(y_train, y_pred_train)
mse_test = MSE(y_test, y_pred_test)
#RMSEの算出
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
#RMSEの表示
print('訓練データに対しては'+ ' ' + str(rmse_train))
print('評価データに対しては'+ ' ' + str(rmse_test))

訓練データに対しては 6.217809054834282
評価データに対しては 11.880373768630049


In [988]:
model3.fit(X_train,y_train)

RandomForestRegressor(n_estimators=100000, random_state=50)

In [989]:
#予測値の算出
y_pred_train = model3.predict(X_train)
y_pred_test = model3.predict(X_test)
#MSEの算出
mse_train = MSE(y_train, y_pred_train)
mse_test = MSE(y_test, y_pred_test)
#RMSEの算出
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
#RMSEの表示
print('訓練データに対しては'+ ' ' + str(rmse_train))
print('評価データに対しては'+ ' ' + str(rmse_test))

訓練データに対しては 6.213851828745285
評価データに対しては 11.743810531328458


### 時系列分割で精度を確認する

In [ ]:
from sklearn.model_selsction import TimeSeriesSplit